In [1]:
# Nice Formatting within Jupyter Notebook
%matplotlib inline
#from IPython.display import display # Allows multiple displays from a single code-cell

# System functionality
import sys
sys.path.append('..')

# Import Here any Additional modules you use. To import utilities we provide, use something like:
#   from utils.plotter import plot_hinton

# Your Code goes here:
#from __future__ import division, print_function # Imports from __future__ since we're running Python 2
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split #, KFold
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
#from sklearn import preprocessing
from numpy import unravel_index
from matplotlib.colors import ListedColormap
from numpy import linspace
%matplotlib inline

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.metrics import log_loss
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

In [3]:
#####################
###   load data   ###
#####################

# load datasets
data_path = "/home/philipp/Documents/boku/remote_sensing_and_image_processing/final_ass/train_data_LAI_ext.csv"
train = pd.read_csv(data_path, delimiter = ',', index_col=0)
data_path = "/home/philipp/Documents/boku/remote_sensing_and_image_processing/final_ass/test_data.csv"
test = pd.read_csv(data_path, delimiter = ',', index_col=0)

train = shuffle(train, random_state=399)

X_train = train.drop(['crop_id','geometry'], axis=1).values
y_train = train['crop_id'].values
X_test = test.drop(['crop_id','geometry'], axis=1).values
y_test = test['crop_id'].values

print("Train: number of instances: {}, number of attributes: {}".format(train.shape[0], train.shape[1]))
print("Test: number of instances: {}, number of attributes: {}".format(test.shape[0], test.shape[1]))

Train: number of instances: 23927, number of attributes: 101
Test: number of instances: 4584, number of attributes: 101


In [4]:
##########################
###   exploring data   ###
##########################

print(train.head(7))

train.describe()

       crop_id                                     geometry  B02_0322  \
3176         2  POINT (637759.1889929901 5347053.846079555)       612   
12822       10   POINT (629080.4447021568 5353786.99907408)       499   
10535        5  POINT (635432.9099537274 5347895.686695563)       978   
4295         6  POINT (621032.3637216241 5336976.096246178)       439   
4419         6  POINT (620882.5791343282 5335696.427355875)       485   
19420       16    POINT (620427.96690282 5361817.013792816)       515   
10754        5  POINT (616312.3717091422 5337878.037065835)       547   

       B03_0322  B04_0322  B05_0322  B06_0322  B07_0322  B08A_0322  B08_0322  \
3176        786      1103      1294      1436      1595       1898      1785   
12822       598       859       962      1059      1151       1329      1263   
10535      1193      1609      1776      1910      2018       2192      2140   
4295        506       626       834      1006      1038       1178       954   
4419        538

,crop_id,B02_0322,B03_0322,B04_0322,B05_0322,B06_0322,B07_0322,B08A_0322,B08_0322,B11_0322,...,B03_0920,B04_0920,B05_0920,B06_0920,B07_0920,B08A_0920,B08_0920,B11_0920,B12_0920,LAI_0920
count,23927.000000,23927.000000,23927.000000,23927.000000,23927.000000,23927.000000,23927.000000,23927.000000,23927.000000,23927.000000,...,23927.000000,23927.000000,23927.000000,23927.000000,23927.000000,23927.000000,23927.000000,23927.000000,23927.000000,23927.000000
mean,8.482050,581.711289,708.570569,868.689514,1105.141806,1406.050612,1542.769048,1709.121829,1627.660384,2083.109249,...,1036.234756,1145.357964,1533.938145,2205.859322,2423.168722,2644.641367,2453.345844,2783.795419,2290.480503,522.687090
std,4.884215,111.302379,145.294160,192.816073,229.600579,474.906125,528.019260,549.404809,553.329138,386.483592,...,329.360170,478.369615,432.459862,776.586097,897.747254,900.895960,894.823978,573.800521,722.710582,692.705114
min,1.000000,279.000000,295.000000,375.000000,440.000000,497.000000,552.000000,665.000000,612.000000,987.000000,...,329.000000,227.000000,535.000000,734.000000,764.000000,891.000000,719.000000,1048.000000,478.000000,-223.000000
25%,3.000000,509.000000,616.000000,748.000000,946.000000,1078.500000,1184.000000,1339.000000,1252.000000,1829.000000,...,795.000000,791.000000,1225.000000,1594.000000,1724.000000,1928.000000,1747.000000,2428.000000,1730.000000,113.000000
50%,8.000000,569.000000,693.000000,847.000000,1084.000000,1287.000000,1409.000000,1586.000000,1492.000000,2113.000000,...,956.000000,1125.000000,1431.000000,2057.000000,2232.000000,2467.000000,2265.000000,2735.000000,2393.000000,260.000000
75%,12.000000,635.000000,782.000000,965.000000,1245.000000,1627.000000,1772.500000,1951.000000,1862.000000,2349.000000,...,1186.000000,1419.000000,1746.000000,2657.000000,2932.000000,3180.500000,2963.000000,3118.000000,2765.000000,671.000000
max,16.000000,1486.000000,1551.000000,1963.000000,2141.000000,3945.000000,4418.000000,4598.000000,4544.000000,3944.000000,...,2948.000000,3399.000000,3763.000000,5363.000000,6430.000000,6575.000000,6203.000000,5149.000000,4936.000000,5760.000000


In [5]:
############################
###   standardize data   ###
############################

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
##########################
##########################
###                    ###
###   Classification   ###
###                    ###
##########################
##########################

In [ ]:
##########################################
###   RFC - Random Forest Classifier   ###
##########################################

In [9]:
clf = RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=399, n_jobs=8)
scores = cross_val_score(clf, X_train, y_train, cv=10)
print(np.mean(scores), scores)

0.7816666666666667 [0.775      0.80833333 0.81666667 0.775      0.775      0.75833333
 0.81666667 0.75       0.79166667 0.75      ]


In [6]:
clf = RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=399, n_jobs=8)
scores = cross_val_score(clf, X_train, y_train, cv=10)
print(np.mean(scores), scores)

0.9662308509909756 [0.97033013 0.96698705 0.96155453 0.9653155  0.96030088 0.96573339
 0.96782282 0.96906355 0.96948161 0.96571906]


In [10]:
param_grid = [
    {'n_estimators': [1, 10, 100, 1000], 'criterion': ['gini', 'entropy'], \
     'max_depth': [None, 10, 100, 1000], 'max_features': ['sqrt', 'log2', None], },
 ]

In [11]:
clf = GridSearchCV(RandomForestClassifier(), param_grid, cv=10, n_jobs=8)
clf.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestClassifier(), n_jobs=8,
             param_grid=[{'criterion': ['gini', 'entropy'],
                          'max_depth': [None, 10, 100, 1000],
                          'max_features': ['sqrt', 'log2', None],
                          'n_estimators': [1, 10, 100, 1000]}])

In [12]:
print("------------------------------------------------------------------------------------------------------")
print()
print("Best parameters set found on development set: ", clf.best_params_)
print()
print('Best mean cross-validated score:', clf.best_score_)
print()
print("------------------------------------------------------------------------------------------------------")
print("Grid scores for SVM:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print("------------------------------------------------------------------------------------------------------")

------------------------------------------------------------------------------------------------------

Best parameters set found on development set:  {'criterion': 'gini', 'max_depth': 1000, 'max_features': 'sqrt', 'n_estimators': 1000}

Best mean cross-validated score: 0.8391666666666666

------------------------------------------------------------------------------------------------------
Grid scores for SVM:

0.600 (+/-0.076) for {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'n_estimators': 1}
0.773 (+/-0.043) for {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'n_estimators': 10}
0.824 (+/-0.053) for {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'n_estimators': 100}
0.837 (+/-0.061) for {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'n_estimators': 1000}
0.592 (+/-0.095) for {'criterion': 'gini', 'max_depth': None, 'max_features': 'log2', 'n_estimators': 1}
0.774 (+/-0.051) for {'criterion': 'gini', 'max_dept

In [6]:
param_grid = [
    {'n_estimators': [600,1000,3000], 'criterion': ['gini'], \
     'max_depth': [1000], 'max_features': ['sqrt'], },
 ]

In [7]:
clf = GridSearchCV(RandomForestClassifier(), param_grid, cv=10, n_jobs=8)
clf.fit(X_train, y_train)

KeyboardInterrupt: 

In [21]:
print("------------------------------------------------------------------------------------------------------")
print()
print("Best parameters set found on development set: ", clf.best_params_)
print()
print('Best mean cross-validated score:', clf.best_score_)
print()
print("------------------------------------------------------------------------------------------------------")
print("Grid scores for SVM:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print("------------------------------------------------------------------------------------------------------")

------------------------------------------------------------------------------------------------------

Best parameters set found on development set:  {'criterion': 'gini', 'max_depth': 1000, 'max_features': 'sqrt', 'n_estimators': 600}

Best mean cross-validated score: 0.8358333333333332

------------------------------------------------------------------------------------------------------
Grid scores for SVM:

0.836 (+/-0.045) for {'criterion': 'gini', 'max_depth': 1000, 'max_features': 'sqrt', 'n_estimators': 600}
0.833 (+/-0.061) for {'criterion': 'gini', 'max_depth': 1000, 'max_features': 'sqrt', 'n_estimators': 1000}
0.835 (+/-0.062) for {'criterion': 'gini', 'max_depth': 1000, 'max_features': 'sqrt', 'n_estimators': 3000}
------------------------------------------------------------------------------------------------------


In [ ]:
###########################################
###   SVC - Support Vector Classifier   ###
###########################################

In [5]:
clf = SVC(kernel='linear', C=1, random_state=399)
scores = cross_val_score(clf, X_train, y_train, cv=10, n_jobs=8)
print(np.mean(scores), scores)

0.8541666666666667 [0.825      0.86666667 0.85833333 0.875      0.83333333 0.85833333
 0.85833333 0.86666667 0.86666667 0.83333333]


In [8]:
clf = SVC(kernel='linear', C=1, random_state=399)
scores = cross_val_score(clf, X_train, y_train, cv=10, n_jobs=8)
print(np.mean(scores), scores)

0.8799999999999999 [0.88333333 0.9        0.86666667 0.90833333 0.88333333 0.90833333
 0.85833333 0.85       0.88333333 0.85833333]


In [7]:
clf = SVC(kernel='linear', C=1, random_state=399)
scores = cross_val_score(clf, X_train, y_train, cv=10, n_jobs=8)
print(np.mean(scores), scores)

0.9811511452718141 [0.98328458 0.98203092 0.98203092 0.97743418 0.97785207 0.98161304
 0.98035938 0.98620401 0.97993311 0.98076923]


In [6]:
param_grid = [
    {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
    {'C': [1, 10, 100, 1000], 'gamma': ['scale', 'auto'], 'kernel': ['rbf']},
    {'C': [1, 10, 100, 1000], 'gamma': ['scale', 'auto'], 'degree': [2, 3, 4], 'kernel': ['poly']},
    {'C': [1, 10, 100, 1000], 'gamma': ['scale', 'auto'], 'kernel': ['sigmoid']},
 ]

In [7]:
clf = GridSearchCV(SVC(), param_grid, cv=10, n_jobs=8)
clf.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=SVC(), n_jobs=8,
             param_grid=[{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
                         {'C': [1, 10, 100, 1000], 'gamma': ['scale', 'auto'],
                          'kernel': ['rbf']},
                         {'C': [1, 10, 100, 1000], 'degree': [2, 3, 4],
                          'gamma': ['scale', 'auto'], 'kernel': ['poly']},
                         {'C': [1, 10, 100, 1000], 'gamma': ['scale', 'auto'],
                          'kernel': ['sigmoid']}])

In [8]:
print("------------------------------------------------------------------------------------------------------")
print()
print("Best parameters set found on development set: ", clf.best_params_)
print()
print('Best mean cross-validated score:', clf.best_score_)
print()
print("------------------------------------------------------------------------------------------------------")
print("Grid scores for SVM:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))

------------------------------------------------------------------------------------------------------

Best parameters set found on development set:  {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}

Best mean cross-validated score: 0.9883333333333335

------------------------------------------------------------------------------------------------------
Grid scores for SVM:

0.971 (+/-0.009) for {'C': 1, 'kernel': 'linear'}
0.972 (+/-0.010) for {'C': 10, 'kernel': 'linear'}
0.968 (+/-0.010) for {'C': 100, 'kernel': 'linear'}
0.968 (+/-0.009) for {'C': 1000, 'kernel': 'linear'}
0.971 (+/-0.007) for {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}
0.971 (+/-0.007) for {'C': 1, 'gamma': 'auto', 'kernel': 'rbf'}
0.988 (+/-0.005) for {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
0.988 (+/-0.005) for {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}
0.988 (+/-0.005) for {'C': 100, 'gamma': 'scale', 'kernel': 'rbf'}
0.988 (+/-0.005) for {'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}
0.988 (+/-0.005) for {'C':

In [12]:
######################
###   Prediction   ###
######################

In [13]:
# prediction with the best preforming model
clf = SVC(kernel='rbf', C=10, gamma='scale', random_state=399)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)

In [14]:
pred = clf.predict(X_test)

In [17]:
confusion_matrix(y_test, pred)

array([[355,   1,   1,  15,   2,  13,   3,   5,   0,   2,   2,   1],
       [  4, 372,   0,  18,   2,   0,   1,   0,   0,   3,   0,   0],
       [  0,   0, 380,   2,   0,   0,   0,   0,   8,   0,   9,   1],
       [ 12,   5,   5, 317,  14,   2,   3,   3,   0,   9,  23,   7],
       [  5,   7,   0,  18, 314,   4,  10,  21,   1,   7,  13,   0],
       [ 11,   3,   0,  11,   4, 362,   3,   2,   0,   3,   1,   0],
       [  1,   0,   0,   6,   3,   6, 374,   2,   0,   2,   1,   5],
       [  3,   0,   0,   7,  35,   1,   5, 346,   0,   3,   0,   0],
       [  0,   0,   9,   3,   0,   0,   0,   0, 374,   3,   9,   2],
       [  0,   2,   0,   3,   2,   1,   0,   3,   0, 372,   3,  14],
       [  3,   0,   6,   3,   3,   0,   0,   0,   1,   6, 358,   0],
       [  0,   0,   0,   1,   0,   0,   0,   0,   0,   4,   0, 199]])

In [18]:
accuracy_score(y_test, pred)

0.8994328097731239

In [9]:
# prediction with the best preforming model
clf = SVC(kernel='rbf', C=10, gamma='scale', random_state=399)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)

In [10]:
pred = clf.predict(X_test)

In [11]:
confusion_matrix(y_test, pred)

array([[375,   0,   2,   6,   1,   8,   2,   3,   0,   1,   1,   1],
       [  3, 384,   0,   8,   2,   0,   1,   0,   0,   2,   0,   0],
       [  0,   0, 379,   3,   1,   1,   0,   0,  13,   0,   3,   0],
       [ 12,  12,   5, 320,  14,   2,   3,   6,   0,   9,  14,   3],
       [  3,  10,   0,  11, 334,   2,  10,  12,   0,   4,  14,   0],
       [ 12,   1,   0,   8,   4, 369,   2,   1,   0,   2,   1,   0],
       [  0,   0,   0,   4,   5,   2, 384,   2,   0,   3,   0,   0],
       [  1,   1,   0,   1,  20,   0,   1, 374,   0,   0,   2,   0],
       [  0,   0,   4,   4,   0,   0,   0,   0, 380,   6,   5,   1],
       [  1,   1,   0,   1,   0,   0,   0,   3,   0, 383,   3,   8],
       [  3,   0,   1,   2,   4,   0,   0,   0,   0,   1, 369,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 204]])

In [12]:
accuracy_score(y_test, pred)

0.9282286212914486

In [8]:
# prediction with the best preforming model
clf = SVC(kernel='rbf', C=10, gamma='scale', random_state=399)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)

In [9]:
confusion_matrix(y_test, pred)

array([[375,   0,   2,   4,   1,  11,   1,   3,   0,   1,   1,   1],
       [  4, 384,   0,   6,   2,   0,   1,   1,   0,   2,   0,   0],
       [  0,   0, 385,   2,   1,   2,   0,   0,   6,   1,   3,   0],
       [ 17,  12,   6, 319,  15,   2,   1,   6,   1,   4,  15,   2],
       [  2,   9,   1,   9, 338,   1,   6,  12,   0,   5,  17,   0],
       [ 15,   1,   0,   9,   3, 366,   2,   1,   1,   1,   1,   0],
       [  0,   0,   0,   3,   3,   2, 386,   2,   0,   4,   0,   0],
       [  1,   0,   0,   3,  19,   0,   4, 366,   0,   1,   6,   0],
       [  0,   0,   4,   3,   0,   0,   0,   0, 379,   6,   6,   2],
       [  2,   0,   0,   2,   0,   1,   0,   2,   0, 381,   4,   8],
       [  3,   0,   1,   0,   5,   0,   0,   0,   0,   0, 371,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   1,   0, 203]])

In [10]:
accuracy_score(y_test, pred)

0.9277923211169284

In [11]:
X_train.shape

(23927, 99)